## 1. How to import PySpark and check the version?

In [1]:
import findspark
findspark.init()

# Creating a SparkSession: A SparkSession is the entry point for using the PySpark DataFrame and SQL API.
# To create a SparkSession, use the following code
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark 101 Exercises").getOrCreate()

# Get version details
print(spark.version)

4.0.1


## 2. How to convert the index of a PySpark DataFrame into a column?

In [2]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id

# Assuming df is your DataFrame
df = spark.createDataFrame([
("Alice", 1),
("Bob", 2),
("Charlie", 3),
], ["Name", "Value"])

# Define window specification
w = Window.orderBy(monotonically_increasing_id())

# Add index
df = df.withColumn("index", row_number().over(w) - 1)

df.show()

+-------+-----+-----+
|   Name|Value|index|
+-------+-----+-----+
|  Alice|    1|    0|
|    Bob|    2|    1|
|Charlie|    3|    2|
+-------+-----+-----+



## 3. How to combine many lists to form a PySpark DataFrame?

In [3]:
list1 = ["a", "b", "c", "d"]
list2 = [1, 2, 3, 4]
# Create an RDD from the lists and convert it to a DataFrame
rdd = spark.sparkContext.parallelize(list(zip(list1, list2)))
df = rdd.toDF(["Column1", "Column2"])

# Show the DataFrame
df.show()

+-------+-------+
|Column1|Column2|
+-------+-------+
|      a|      1|
|      b|      2|
|      c|      3|
|      d|      4|
+-------+-------+



## 4. How to get the items of list A not present in list B?

In [4]:
sc = spark.sparkContext

list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]
# Convert lists to RDD
rdd_A = sc.parallelize(list_A)
rdd_B = sc.parallelize(list_B)

# Perform subtract operation
result_rdd = rdd_A.subtract(rdd_B)

# Collect result
result_list = result_rdd.collect()
print(result_list)

[1, 2, 3]


## 5. How to get the items not common to both list A and list B?

In [5]:
sc = spark.sparkContext

list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

# Convert lists to RDD
rdd_A = sc.parallelize(list_A)
rdd_B = sc.parallelize(list_B)

# Perform subtract operation
result_rdd_A = rdd_A.subtract(rdd_B)
result_rdd_B = rdd_B.subtract(rdd_A)

# Union the two RDDs
result_rdd = result_rdd_A.union(result_rdd_B)

# Collect result
result_list = result_rdd.collect()

print(result_list)

[1, 2, 3, 6, 7, 8]


## 6. How to get the minimum, 25th percentile, median, 75th, and max of a numeric column?

In [6]:
# Create a sample DataFrame
data = [("A", 10), ("B", 20), ("C", 30), ("D", 40), ("E", 50), ("F", 15), ("G", 28), ("H", 54), ("I", 41), ("J", 86)]
df = spark.createDataFrame(data, ["Name", "Age"])

df.show()

# Calculate percentiles
quantiles = df.approxQuantile("Age", [0.0, 0.25, 0.5, 0.75, 1.0], 0.01)

print("Min: ", quantiles[0])
print("25th percentile: ", quantiles[1])
print("Median: ", quantiles[2])
print("75th percentile: ", quantiles[3])
print("Max: ", quantiles[4])

+----+---+
|Name|Age|
+----+---+
|   A| 10|
|   B| 20|
|   C| 30|
|   D| 40|
|   E| 50|
|   F| 15|
|   G| 28|
|   H| 54|
|   I| 41|
|   J| 86|
+----+---+

Min:  10.0
25th percentile:  20.0
Median:  30.0
75th percentile:  50.0
Max:  86.0


## 7. How to get frequency counts of unique items of a column?

In [7]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()
df.groupBy("job").count().show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+

+---------+-----+
|      job|count|
+---------+-----+
| Engineer|    4|
|Scientist|    2|
|   Doctor|    1|
+---------+-----+



## 8. How to keep only top 2 most frequent values as it is and replace everything else as ‘Other’?

In [8]:
from pyspark.sql import Row
from pyspark.sql.functions import col, when

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()

# Get the top 2 most frequent jobs
top_2_jobs = df.groupBy('job').count().orderBy('count', ascending=False).limit(2).select('job').rdd.flatMap(lambda x: x).collect()

# Replace all but the top 2 most frequent jobs with 'Other'
df = df.withColumn('job', when(col('job').isin(top_2_jobs), col('job')).otherwise('Other'))

# show DataFrame
df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|    Other|
+----+---------+



## 9. How to Drop rows with NA values specific to a particular column?

In [9]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()

df_2 = df.dropna(subset=['Value'])

df_2.show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|NULL|
|   B| NULL| 123|
|   B|    3| 456|
|   D| NULL|NULL|
+----+-----+----+

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|NULL|
|   B|    3| 456|
+----+-----+----+



## 10. How to rename columns of a PySpark DataFrame using two lists – one containing the old column names and the other containing the new column names?

In [10]:
# suppose you have the following DataFrame
df = spark.createDataFrame([(1, 2, 3), (4, 5, 6)], ["col1", "col2", "col3"])

# old column names
old_names = ["col1", "col2", "col3"]

# new column names
new_names = ["new_col1", "new_col2", "new_col3"]

df.show()

# renaming
for old_name, new_name in zip(old_names, new_names):
    df = df.withColumnRenamed(old_name, new_name)

df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
+----+----+----+

+--------+--------+--------+
|new_col1|new_col2|new_col3|
+--------+--------+--------+
|       1|       2|       3|
|       4|       5|       6|
+--------+--------+--------+



## 11. How to bin a numeric list to 10 groups of equal size?

In [11]:
from pyspark.sql.functions import rand
from pyspark.ml.feature import Bucketizer

# Create a DataFrame with a single column "values" filled with random numbers
num_items = 100
df = spark.range(num_items).select(rand(seed=42).alias("values"))

df.show(5)
# Define the bucket boundaries
num_buckets = 10
quantiles = df.stat.approxQuantile("values", [i/num_buckets for i in range(num_buckets+1)], 0.01)

# Create the Bucketizer
bucketizer = Bucketizer(splits=quantiles, inputCol="values", outputCol="buckets")

# Apply the Bucketizer
df_buck = bucketizer.transform(df)

#Frequency table
df_buck.groupBy("buckets").count().show()

# Show the original and bucketed values
df_buck.show(5)

+-------------------+
|             values|
+-------------------+
|  0.619189370225301|
| 0.5096018842446481|
| 0.8325259388871524|
|0.26322809041172357|
| 0.6702867696264135|
+-------------------+
only showing top 5 rows
+-------+-----+
|buckets|count|
+-------+-----+
|    8.0|   10|
|    0.0|    8|
|    7.0|   10|
|    4.0|   10|
|    3.0|   10|
|    2.0|   10|
|    6.0|   10|
|    5.0|   10|
|    9.0|   12|
|    1.0|   10|
+-------+-----+

+-------------------+-------+
|             values|buckets|
+-------------------+-------+
|  0.619189370225301|    5.0|
| 0.5096018842446481|    4.0|
| 0.8325259388871524|    8.0|
|0.26322809041172357|    2.0|
| 0.6702867696264135|    6.0|
+-------------------+-------+
only showing top 5 rows


## 12. How to create contigency table?

In [12]:
# Example DataFrame
data = [("A", "X"), ("A", "Y"), ("A", "X"), ("B", "Y"), ("B", "X"), ("C", "X"), ("C", "X"), ("C", "Y")]
df = spark.createDataFrame(data, ["category1", "category2"])

df.show()
# Frequency
df.cube("category1").count().show()

# Contingency table
df.crosstab('category1', 'category2').show()

+---------+---------+
|category1|category2|
+---------+---------+
|        A|        X|
|        A|        Y|
|        A|        X|
|        B|        Y|
|        B|        X|
|        C|        X|
|        C|        X|
|        C|        Y|
+---------+---------+

+---------+-----+
|category1|count|
+---------+-----+
|     NULL|    8|
|        A|    3|
|        B|    2|
|        C|    3|
+---------+-----+

+-------------------+---+---+
|category1_category2|  X|  Y|
+-------------------+---+---+
|                  B|  1|  1|
|                  C|  2|  1|
|                  A|  2|  1|
+-------------------+---+---+



## 13. How to find the numbers that are multiples of 3 from a column?

In [13]:
from pyspark.sql.functions import rand
from pyspark.sql.functions import col, when


# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

# Assuming df is your DataFrame and "your_column" is the column with the numbers
df = df.withColumn("is_multiple_of_3", when(col("random") % 3 == 0, 1).otherwise(0))

df.show()

+---+------+
| id|random|
+---+------+
|  0|     7|
|  1|     9|
|  2|     7|
|  3|     8|
|  4|     9|
|  5|     8|
|  6|     1|
|  7|     1|
|  8|     7|
|  9|     8|
+---+------+

+---+------+----------------+
| id|random|is_multiple_of_3|
+---+------+----------------+
|  0|     7|               0|
|  1|     9|               1|
|  2|     7|               0|
|  3|     8|               0|
|  4|     9|               1|
|  5|     8|               0|
|  6|     1|               0|
|  7|     1|               0|
|  8|     7|               0|
|  9|     8|               0|
+---+------+----------------+



## 14. How to extract items at given positions from a column?

In [14]:
from pyspark.sql.functions import rand
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

pos = [0, 4, 8, 5]

# Define window specification
w = Window.orderBy(monotonically_increasing_id())

# Add index
df = df.withColumn("index", row_number().over(w) - 1)

df.show()

# Filter the DataFrame based on the specified positions
df_filtered = df.filter(df.index.isin(pos))

df_filtered.show()

+---+------+
| id|random|
+---+------+
|  0|     7|
|  1|     9|
|  2|     7|
|  3|     8|
|  4|     9|
|  5|     8|
|  6|     1|
|  7|     1|
|  8|     7|
|  9|     8|
+---+------+

+---+------+-----+
| id|random|index|
+---+------+-----+
|  0|     7|    0|
|  1|     9|    1|
|  2|     7|    2|
|  3|     8|    3|
|  4|     9|    4|
|  5|     8|    5|
|  6|     1|    6|
|  7|     1|    7|
|  8|     7|    8|
|  9|     8|    9|
+---+------+-----+

+---+------+-----+
| id|random|index|
+---+------+-----+
|  0|     7|    0|
|  4|     9|    4|
|  5|     8|    5|
|  8|     7|    8|
+---+------+-----+



## 15. How to stack two DataFrames vertically ?

In [15]:
# Create DataFrame for region A
df_A = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 10), ("orange", 2, 8)], ["Name", "Col_1", "Col_2"])
df_A.show()

# Create DataFrame for region B
df_B = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 15), ("grape", 4, 6)], ["Name", "Col_1", "Col_3"])
df_B.show()

df_A.union(df_B).show()

+------+-----+-----+
|  Name|Col_1|Col_2|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   10|
|orange|    2|    8|
+------+-----+-----+

+------+-----+-----+
|  Name|Col_1|Col_3|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   15|
| grape|    4|    6|
+------+-----+-----+

+------+-----+-----+
|  Name|Col_1|Col_2|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   10|
|orange|    2|    8|
| apple|    3|    5|
|banana|    1|   15|
| grape|    4|    6|
+------+-----+-----+



## 16. How to compute the mean squared error on a truth and predicted columns?

In [16]:
# Assume you have a DataFrame df with two columns "actual" and "predicted"
# For the sake of example, we'll create a sample DataFrame
data = [(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]
df = spark.createDataFrame(data, ["actual", "predicted"])

df.show()
# Calculate the squared differences
df = df.withColumn("squared_error", pow((col("actual") - col("predicted")), 2))

# Calculate the mean squared error
mse = df.agg({"squared_error": "avg"}).collect()[0][0]

print(f"Mean Squared Error (MSE) = {mse}")

+------+---------+
|actual|predicted|
+------+---------+
|     1|        1|
|     2|        4|
|     3|        9|
|     4|       16|
|     5|       25|
+------+---------+

Mean Squared Error (MSE) = 116.8


## 17. How to convert the first character of each element in a series to uppercase?

In [17]:
from pyspark.sql.functions import initcap
# Suppose you have the following DataFrame
data = [("john",), ("alice",), ("bob",)]
df = spark.createDataFrame(data, ["name"])

df.show()
# Convert the first character to uppercase
df = df.withColumn("name", initcap(df["name"]))

df.show()

+-----+
| name|
+-----+
| john|
|alice|
|  bob|
+-----+

+-----+
| name|
+-----+
| John|
|Alice|
|  Bob|
+-----+



## 18. How to compute summary statistics for all columns in a dataframe

In [18]:
# For the sake of example, we'll create a sample DataFrame
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df = spark.createDataFrame(data, ["name", "age" , "salary"])

df.show()
# Summary statistics
summary = df.summary()

# Show the summary statistics
summary.show()

+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  James| 34| 55000|
|Michael| 30| 70000|
| Robert| 37| 60000|
|  Maria| 29| 80000|
|    Jen| 32| 65000|
+-------+---+------+

+-------+------+-----------------+-----------------+
|summary|  name|              age|           salary|
+-------+------+-----------------+-----------------+
|  count|     5|                5|                5|
|   mean|  NULL|             32.4|          66000.0|
| stddev|  NULL|3.209361307176242|9617.692030835675|
|    min| James|               29|            55000|
|    25%|  NULL|               30|            60000|
|    50%|  NULL|               32|            65000|
|    75%|  NULL|               34|            70000|
|    max|Robert|               37|            80000|
+-------+------+-----------------+-----------------+



## 19. How to calculate the number of characters in each word in a column?

In [19]:
from pyspark.sql import functions as F

# Suppose you have the following DataFrame
data = [("john",), ("alice",), ("bob",)]
df = spark.createDataFrame(data, ["name"])

df.show()

df = df.withColumn('word_length', F.length(df.name))
df.show()

+-----+
| name|
+-----+
| john|
|alice|
|  bob|
+-----+

+-----+-----------+
| name|word_length|
+-----+-----------+
| john|          4|
|alice|          5|
|  bob|          3|
+-----+-----------+



## 20 How to compute difference of differences between consecutive numbers of a column?

In [20]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# For the sake of example, we'll create a sample DataFrame
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df = spark.createDataFrame(data, ["name", "age" , "salary"])

df.show()

# Define window specification
df = df.withColumn("id", F.monotonically_increasing_id())
window = Window.orderBy("id")

# Generate the lag of the variable
df = df.withColumn("prev_value", F.lag(df.salary).over(window))

# Compute the difference with lag
df = df.withColumn("diff", F.when(F.isnull(df.salary - df.prev_value), 0)
.otherwise(df.salary - df.prev_value)).drop("id")

df.show()

+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  James| 34| 55000|
|Michael| 30| 70000|
| Robert| 37| 60000|
|  Maria| 29| 80000|
|    Jen| 32| 65000|
+-------+---+------+

+-------+---+------+----------+------+
|   name|age|salary|prev_value|  diff|
+-------+---+------+----------+------+
|  James| 34| 55000|      NULL|     0|
|Michael| 30| 70000|     55000| 15000|
| Robert| 37| 60000|     70000|-10000|
|  Maria| 29| 80000|     60000| 20000|
|    Jen| 32| 65000|     80000|-15000|
+-------+---+------+----------+------+



## 21. How to get the day of month, week number, day of year and day of week from a date strings?

In [21]:
from pyspark.sql.functions import to_date, dayofmonth, weekofyear, dayofyear, dayofweek
# example data
data = [("2023-05-18","01 Jan 2010",), ("2023-12-31", "01 Jan 2010",)]
df = spark.createDataFrame(data, ["date_str_1", "date_str_2"])

df.show()

# Convert date string to date format
df = df.withColumn("date_1", to_date(df.date_str_1, 'yyyy-MM-dd'))
df = df.withColumn("date_2", to_date(df.date_str_2, 'dd MMM yyyy'))

df = df.withColumn("day_of_month", dayofmonth(df.date_1))\
.withColumn("week_number", weekofyear(df.date_1))\
.withColumn("day_of_year", dayofyear(df.date_1))\
.withColumn("day_of_week", dayofweek(df.date_1))

df.show()

+----------+-----------+
|date_str_1| date_str_2|
+----------+-----------+
|2023-05-18|01 Jan 2010|
|2023-12-31|01 Jan 2010|
+----------+-----------+

+----------+-----------+----------+----------+------------+-----------+-----------+-----------+
|date_str_1| date_str_2|    date_1|    date_2|day_of_month|week_number|day_of_year|day_of_week|
+----------+-----------+----------+----------+------------+-----------+-----------+-----------+
|2023-05-18|01 Jan 2010|2023-05-18|2010-01-01|          18|         20|        138|          5|
|2023-12-31|01 Jan 2010|2023-12-31|2010-01-01|          31|         52|        365|          1|
+----------+-----------+----------+----------+------------+-----------+-----------+-----------+



## 22. How to convert year-month string to dates corresponding to the 4th day of the month?

In [22]:
from pyspark.sql.functions import expr, col
# example dataframe
df = spark.createDataFrame([('Jan 2010',), ('Feb 2011',), ('Mar 2012',)], ['MonthYear'])

df.show()

# convert YearMonth to date (default to first day of the month)
df = df.withColumn('Date', expr("to_date(MonthYear, 'MMM yyyy')"))

df.show()

# replace day with 4
df = df.withColumn('Date', expr("date_add(date_sub(Date, day(Date) - 1), 3)"))

df.show()

+---------+
|MonthYear|
+---------+
| Jan 2010|
| Feb 2011|
| Mar 2012|
+---------+

+---------+----------+
|MonthYear|      Date|
+---------+----------+
| Jan 2010|2010-01-01|
| Feb 2011|2011-02-01|
| Mar 2012|2012-03-01|
+---------+----------+

+---------+----------+
|MonthYear|      Date|
+---------+----------+
| Jan 2010|2010-01-04|
| Feb 2011|2011-02-04|
| Mar 2012|2012-03-04|
+---------+----------+



## 23 How to filter words that contain atleast 2 vowels from a series?

In [23]:
from pyspark.sql.functions import col, length, translate
# example dataframe
df = spark.createDataFrame([('Apple',), ('Orange',), ('Plan',) , ('Python',) , ('Money',)], ['Word'])

df.show()

# Filter words that contain at least 2 vowels
df_filtered = df.where((length(col('Word')) - length(translate(col('Word'), 'AEIOUaeiou', ''))) >= 2)
df_filtered.show()

+------+
|  Word|
+------+
| Apple|
|Orange|
|  Plan|
|Python|
| Money|
+------+

+------+
|  Word|
+------+
| Apple|
|Orange|
| Money|
+------+



## 24. How to filter valid emails from a list?

In [24]:
# Create a list
data = ['buying books at amazom.com', 'rameses@egypt.com', 'matt@t.co', 'narendra@modi.com']

# Convert the list to DataFrame
df = spark.createDataFrame(data, "string")
df.show(truncate =False)

# Define a regular expression pattern for emails
pattern = "^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"

# Apply filter operation to keep only valid emails
df_filtered = df.filter(F.col("value").rlike(pattern))

# Show the DataFrame
df_filtered.show()

+--------------------------+
|value                     |
+--------------------------+
|buying books at amazom.com|
|rameses@egypt.com         |
|matt@t.co                 |
|narendra@modi.com         |
+--------------------------+

+-----------------+
|            value|
+-----------------+
|rameses@egypt.com|
|        matt@t.co|
|narendra@modi.com|
+-----------------+



## 25. How to Pivot PySpark DataFrame?

In [25]:
# Sample data
data = [
(2021, 1, "US", 5000),
(2021, 1, "EU", 4000),
(2021, 2, "US", 5500),
(2021, 2, "EU", 4500),
(2021, 3, "US", 6000),
(2021, 3, "EU", 5000),
(2021, 4, "US", 7000),
(2021, 4, "EU", 6000),
]

# Create DataFrame
columns = ["year", "quarter", "region", "revenue"]
df = spark.createDataFrame(data, columns)
df.show()

# Execute the pivot operation
pivot_df = df.groupBy("year", "quarter").pivot("region").sum("revenue")

pivot_df.show()

+----+-------+------+-------+
|year|quarter|region|revenue|
+----+-------+------+-------+
|2021|      1|    US|   5000|
|2021|      1|    EU|   4000|
|2021|      2|    US|   5500|
|2021|      2|    EU|   4500|
|2021|      3|    US|   6000|
|2021|      3|    EU|   5000|
|2021|      4|    US|   7000|
|2021|      4|    EU|   6000|
+----+-------+------+-------+

+----+-------+----+----+
|year|quarter|  EU|  US|
+----+-------+----+----+
|2021|      2|4500|5500|
|2021|      1|4000|5000|
|2021|      3|5000|6000|
|2021|      4|6000|7000|
+----+-------+----+----+



## 26. How to get the mean of a variable grouped by another variable?

In [26]:
from pyspark.sql.functions import mean

# Sample data
data = [("1001", "Laptop", 1000),
("1002", "Mouse", 50),
("1003", "Laptop", 1200),
("1004", "Mouse", 30),
("1005", "Smartphone", 700)]

# Create DataFrame
columns = ["OrderID", "Product", "Price"]
df = spark.createDataFrame(data, columns)

df.show()


# GroupBy and aggregate
result = df.groupBy("Product").agg(mean("Price").alias("Total_Sales"))

# Show results
result.show()

+-------+----------+-----+
|OrderID|   Product|Price|
+-------+----------+-----+
|   1001|    Laptop| 1000|
|   1002|     Mouse|   50|
|   1003|    Laptop| 1200|
|   1004|     Mouse|   30|
|   1005|Smartphone|  700|
+-------+----------+-----+

+----------+-----------+
|   Product|Total_Sales|
+----------+-----------+
|    Laptop|     1100.0|
|     Mouse|       40.0|
|Smartphone|      700.0|
+----------+-----------+



## 27. How to compute the euclidean distance between two columns?

In [27]:
from pyspark.sql.functions import expr
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Define your series
data = [(1, 10), (2, 9), (3, 8), (4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (9, 2), (10, 1)]

# Convert list to DataFrame
df = spark.createDataFrame(data, ["series1", "series2"])

df.show()


# Convert series to vectors
vecAssembler = VectorAssembler(inputCols=["series1", "series2"], outputCol="vectors")
df = vecAssembler.transform(df)

# Calculate squared differences
df = df.withColumn("squared_diff", expr("POW(series1 - series2, 2)"))

# Sum squared differences and take square root
df.agg(expr("SQRT(SUM(squared_diff))").alias("euclidean_distance")).show()

+-------+-------+
|series1|series2|
+-------+-------+
|      1|     10|
|      2|      9|
|      3|      8|
|      4|      7|
|      5|      6|
|      6|      5|
|      7|      4|
|      8|      3|
|      9|      2|
|     10|      1|
+-------+-------+

+------------------+
|euclidean_distance|
+------------------+
| 18.16590212458495|
+------------------+



## 28. How to replace missing spaces in a string with the least frequent character?

In [28]:
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import StringType, ArrayType
from collections import Counter

#Sample DataFrame
df = spark.createDataFrame([('dbc deb abed gade',),], ["string"])
df.show()


def least_freq_char_replace_spaces(s):
    counter = Counter(s.replace(" ", ""))
    least_freq_char = min(counter, key = counter.get)
    return s.replace(' ', least_freq_char)

udf_least_freq_char_replace_spaces = udf(least_freq_char_replace_spaces, StringType())

df = spark.createDataFrame([('dbc deb abed gade',)], ["string"])
df.withColumn('modified_string', udf_least_freq_char_replace_spaces(df['string'])).show()

+-----------------+
|           string|
+-----------------+
|dbc deb abed gade|
+-----------------+

+-----------------+-----------------+
|           string|  modified_string|
+-----------------+-----------------+
|dbc deb abed gade|dbccdebcabedcgade|
+-----------------+-----------------+



## 29. How to create a TimeSeries starting ‘2000-01-01’ and 10 weekends (saturdays) after that having random numbers as values?

In [29]:
from pyspark.sql.functions import expr, explode, sequence, rand

# Start date and end date (start + 10 weekends)
start_date = '2000-01-01'
end_date = '2000-03-04' # Calculated manually: 10 weekends (Saturdays) from start date

# Create a DataFrame with one row containing a sequence from start_date to end_date with a 1 day step
df = spark.range(1).select(
explode(
sequence(
expr(f"date '{start_date}'"),
expr(f"date '{end_date}'"),
expr("interval 1 day")
)
).alias("date")
)

# Filter out the weekdays (retain weekends)
df = df.filter(expr("dayofweek(date) = 7")) # 7 corresponds to Saturday in Spark

# Add the random numbers column
#df = df.withColumn("random_numbers", rand()*10)
df = df.withColumn("random_numbers", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

+----------+--------------+
|      date|random_numbers|
+----------+--------------+
|2000-01-01|             7|
|2000-01-08|             8|
|2000-01-15|            10|
|2000-01-22|             1|
|2000-01-29|             2|
|2000-02-05|             9|
|2000-02-12|             4|
|2000-02-19|             9|
|2000-02-26|             9|
|2000-03-04|             7|
+----------+--------------+



## 30. How to get the nrows, ncolumns, datatype of a dataframe?

In [4]:
from pyspark import SparkFiles


url = "https://raw.githubusercontent.com/selva86/datasets/master/Churn_Modelling.csv"

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("Churn_Modelling.csv"), header=True, inferSchema=True)

df.show(5, truncate=False)

# For number of rows
nrows = df.count()
print("Number of Rows: ", nrows)

# For number of columns
ncols = len(df.columns)
print("Number of Columns: ", ncols)

# For data types of each column
datatypes = df.dtypes
print("Data types: ", datatypes)

NameError: name 'spark' is not defined

## 31. How to rename a specific columns in a dataframe?

In [5]:
# Suppose you have the following DataFrame
df = spark.createDataFrame([('Alice', 1, 30),('Bob', 2, 35)], ["name", "age", "qty"])

df.show()

# Rename lists for specific columns
old_names = ["qty", "age"]
new_names = ["user_qty", "user_age"]

old_names = ["qty", "age"]
new_names = ["user_qty", "user_age"]

# You can then rename the columns like this:
for old_name, new_name in zip(old_names, new_names):
    df = df.withColumnRenamed(old_name, new_name)

df.show()

NameError: name 'spark' is not defined

## 32. How to check if a dataframe has any missing values and count of missing values in each column?

In [6]:
from pyspark.sql.functions import col, sum

# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()


missing = df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns))
has_missing = any(row.asDict().values() for row in missing.collect())
print(has_missing)

missing_count = missing.collect()[0].asDict()
print(missing_count)

NameError: name 'spark' is not defined

## 33 How to replace missing values of multiple numeric columns with the mean?

In [69]:
from pyspark.ml.feature import Imputer
df = spark.createDataFrame([
("A", 1, None),
("B", None, 123 ),
("B", 3, 456),
("D", 6, None),
], ["Name", "var1", "var2"])

df.show()


column_names = ["var1", "var2"]

# Initialize the Imputer
imputer = Imputer(inputCols= column_names, outputCols= column_names, strategy="mean")

# Fit the Imputer
model = imputer.fit(df)

#Transform the dataset
imputed_df = model.transform(df)

imputed_df.show(5)

Py4JJavaError: An error occurred while calling o1877.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 215.0 failed 1 times, most recent failure: Lost task 0.0 in stage 215.0 (TID 487) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 34. How to change the order of columns of a dataframe?

In [ ]:
# Sample data
data = [("John", "Doe", 30), ("Jane", "Doe", 25), ("Alice", "Smith", 22)]

# Create DataFrame from the data
df = spark.createDataFrame(data, ["First_Name", "Last_Name", "Age"])

# Show the DataFrame
df.show()
new_order = ["Age", "First_Name", "Last_Name"]

# Reorder the columns
df = df.select(*new_order)

# Show the DataFrame with reordered columns
df.show()

Py4JJavaError: An error occurred while calling o1126.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 188.0 failed 1 times, most recent failure: Lost task 0.0 in stage 188.0 (TID 451) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 35. How to format or suppress scientific notations in a PySpark DataFrame?

In [ ]:
from pyspark.sql.functions import format_number
# Assuming you have a DataFrame df and the column you want to format is 'your_column'
df = spark.createDataFrame([(1, 0.000000123), (2, 0.000023456), (3, 0.000345678)], ["id", "your_column"])

df.show()


# Determine the number of decimal places you want
decimal_places = 10

df = df.withColumn("your_column", format_number("your_column", decimal_places))
df.show()

Py4JJavaError: An error occurred while calling o1150.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 189.0 failed 1 times, most recent failure: Lost task 0.0 in stage 189.0 (TID 452) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 36. How to format all the values in a dataframe as percentages?

In [34]:
from pyspark.sql.functions import concat, col, lit

# Sample data
data = [(0.1, .08), (0.2, .06), (0.33, .02)]
df = spark.createDataFrame(data, ["numbers_1", "numbers_2"])

df.show()

columns = ["numbers_1", "numbers_2"]
for col_name in columns:
    df = df.withColumn(col_name, concat((col(col_name) * 100).cast('decimal(10, 2)'), lit("%")))

df.show()

Py4JJavaError: An error occurred while calling o1088.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 187.0 failed 1 times, most recent failure: Lost task 0.0 in stage 187.0 (TID 450) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 37. How to filter every nth row in a dataframe?

In [35]:
# Sample data
data = [("Alice", 1), ("Bob", 2), ("Charlie", 3), ("Dave", 4), ("Eve", 5),
("Frank", 6), ("Grace", 7), ("Hannah", 8), ("Igor", 9), ("Jack", 10)]

# Create DataFrame
df = spark.createDataFrame(data, ["Name", "Number"])

df.show()

# Define window
window = Window.orderBy(monotonically_increasing_id())

# Add row_number to DataFrame
df = df.withColumn("rn", row_number().over(window))

n = 5 # filter every 5th row

# Filter every nth row
df = df.filter((df.rn % n) == 0)

df.show()

Py4JJavaError: An error occurred while calling o1112.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 188.0 failed 1 times, most recent failure: Lost task 0.0 in stage 188.0 (TID 451) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 38 How to get the row number of the nth largest value in a column?

In [36]:
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, row_number

# Sample Data
data = [
Row(id=1, column1=5),
Row(id=2, column1=8),
Row(id=3, column1=12),
Row(id=4, column1=1),
Row(id=5, column1=15),
Row(id=6, column1=7),
]

df = spark.createDataFrame(data)
df.show()



window = Window.orderBy(desc("column1"))
df = df.withColumn("row_number", row_number().over(window))

n = 3 # We're interested in the 3rd largest value.
row = df.filter(df.row_number == n).first()

if row:
    print("Row number:", row.row_number)
    print("Column value:", row.column1)

Py4JJavaError: An error occurred while calling o1136.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 189.0 failed 1 times, most recent failure: Lost task 0.0 in stage 189.0 (TID 452) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 39. How to get the last n rows of a dataframe with row sum > 100?

In [ ]:
from pyspark.sql import functions as F
from functools import reduce
# Sample data
data = [(10, 25, 70),
(40, 5, 20),
(70, 80, 100),
(10, 2, 60),
(40, 50, 20)]

# Create DataFrame
df = spark.createDataFrame(data, ["col1", "col2", "col3"])

# Display original DataFrame
df.show()

# Add 'row_sum' column
df = df.withColumn('row_sum', reduce(lambda a, b: a+b, [F.col(x) for x in df.columns]))

# Display DataFrame with 'row_sum'
df.show()

# Filter rows where 'row_sum' > 100
df = df.filter(F.col('row_sum') > 100)

# Display filtered DataFrame
df.show()

# Add 'id' column
df = df.withColumn('id', F.monotonically_increasing_id())

# Get the last 2 rows
df_last_2 = df.sort(F.desc('id')).limit(2)

# Display the last 2 rows
df_last_2.show()

Py4JJavaError: An error occurred while calling o1222.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 192.0 failed 1 times, most recent failure: Lost task 0.0 in stage 192.0 (TID 455) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 40. How to create a column containing the minimum by maximum of each row?

In [37]:
from pyspark.sql.functions import udf, array
from pyspark.sql.types import FloatType

# Sample Data
data = [(1, 2, 3), (4, 5, 6), (7, 8, 9), (10, 11, 12)]

# Create DataFrame
df = spark.createDataFrame(data, ["col1", "col2", "col3"])

df.show()


# Define UDF
def min_max_ratio(row):
    return float(min(row)) / max(row)

min_max_ratio_udf = udf(min_max_ratio, FloatType())

# Apply UDF to create new column
df = df.withColumn('min_by_max', min_max_ratio_udf(array(df.columns)))

df.show()

Py4JJavaError: An error occurred while calling o1160.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 190.0 failed 1 times, most recent failure: Lost task 0.0 in stage 190.0 (TID 453) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 41. How to create a column that contains the penultimate value in each row?

In [38]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, IntegerType

data = [(10, 20, 30),
(40, 60, 50),
(80, 70, 90)]

df = spark.createDataFrame(data, ["Column1", "Column2", "Column3"])

df.show()


# Define UDF to sort array in descending order
sort_array_desc = F.udf(lambda arr: sorted(arr), ArrayType(IntegerType()))

# Create array from columns, sort in descending order and get the penultimate value
df = df.withColumn("row_as_array", sort_array_desc(F.array(df.columns)))
df = df.withColumn("Penultimate", df['row_as_array'].getItem(1))
df = df.drop('row_as_array')

df.show()

Py4JJavaError: An error occurred while calling o1184.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 191.0 failed 1 times, most recent failure: Lost task 0.0 in stage 191.0 (TID 454) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 42. How to normalize all columns in a dataframe?

In [39]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col

# create a sample dataframe
data = [(1, 2, 3),
(2, 3, 4),
(3, 4, 5),
(4, 5, 6)]

df = spark.createDataFrame(data, ["Col1", "Col2", "Col3"])

df.show()

# define the list of columns to be normalized
input_cols = ["Col1", "Col2", "Col3"]

# initialize VectorAssembler with input and output column names
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

# transform the data
df_assembled = assembler.transform(df)

# initialize StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

# fit and transform the data
scalerModel = scaler.fit(df_assembled)
df_normalized = scalerModel.transform(df_assembled)

# if you want to drop the original 'features' column
df_normalized = df_normalized.drop('features')

df_normalized.show(truncate=False)

Py4JJavaError: An error occurred while calling o1208.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 192.0 failed 1 times, most recent failure: Lost task 0.0 in stage 192.0 (TID 455) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 43. How to get the positions where values of two columns match?

In [40]:
from pyspark.sql.functions import when
from pyspark.sql.functions import col

# Create sample DataFrame
data = [("John", "John"), ("Lily", "Lucy"), ("Sam", "Sam"), ("Lucy", "Lily")]
df = spark.createDataFrame(data, ["Name1", "Name2"])

df.show()


# Add new column Match to indicate if Name1 and Name2 match
df = df.withColumn("Match", when(col("Name1") == col("Name2"), True).otherwise(False))

# Display DataFrame
df.show()

Py4JJavaError: An error occurred while calling o1232.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 193.0 failed 1 times, most recent failure: Lost task 0.0 in stage 193.0 (TID 456) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 44. How to create lags and leads of a column by group in a dataframe?

In [41]:
from pyspark.sql.functions import lag, lead, to_date
from pyspark.sql.window import Window

# Create a sample DataFrame
data = [("2023-01-01", "Store1", 100),
("2023-01-02", "Store1", 150),
("2023-01-03", "Store1", 200),
("2023-01-04", "Store1", 250),
("2023-01-05", "Store1", 300),
("2023-01-01", "Store2", 50),
("2023-01-02", "Store2", 60),
("2023-01-03", "Store2", 80),
("2023-01-04", "Store2", 90),
("2023-01-05", "Store2", 120)]

df = spark.createDataFrame(data, ["Date", "Store", "Sales"])

df.show()

# Convert the date from string to date type
df = df.withColumn("Date", to_date(df.Date, 'yyyy-MM-dd'))

# Create a Window partitioned by Store, ordered by Date
windowSpec = Window.partitionBy("Store").orderBy("Date")

# Create lag and lead variables
df = df.withColumn("Lag_Sales", lag(df["Sales"]).over(windowSpec))
df = df.withColumn("Lead_Sales", lead(df["Sales"]).over(windowSpec))

df.show()

Py4JJavaError: An error occurred while calling o1256.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 194.0 failed 1 times, most recent failure: Lost task 0.0 in stage 194.0 (TID 457) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 45. How to get the frequency of unique values in the entire dataframe?

In [42]:
from pyspark.sql.functions import col
# Create a numeric DataFrame
data = [(1, 2, 3),
(2, 3, 4),
(1, 2, 3),
(4, 5, 6),
(2, 3, 4)]
df = spark.createDataFrame(data, ["Column1", "Column2", "Column3"])

# Print DataFrame
df.show()


# get column names
columns = df.columns

# stack all columns into a single column
df_single = None

for c in columns:
    if df_single is None:
        df_single = df.select(col(c).alias("single_column"))
else:
    df_single = df_single.union(df.select(col(c).alias("single_column")))

# generate frequency table
frequency_table = df_single.groupBy("single_column").count().orderBy('count', ascending=False)

# show frequency table
frequency_table.show()

Py4JJavaError: An error occurred while calling o1280.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 195.0 failed 1 times, most recent failure: Lost task 0.0 in stage 195.0 (TID 458) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 46. How to replace both the diagonals of dataframe with 0?

In [43]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql.functions import when, col

# Create a numeric DataFrame
data = [(1, 2, 3, 4),
(2, 3, 4, 5),
(1, 2, 3, 4),
(4, 5, 6, 7)]

df = spark.createDataFrame(data, ["col_1", "col_2", "col_3", "col_4"])

# Print DataFrame
df.show()


# Define window specification
w = Window.orderBy(monotonically_increasing_id())

# Add index
df = df.withColumn("id", row_number().over(w) - 1)

df = df.select([when(col("id") == i, 0).otherwise(col("col_"+str(i+1))).alias("col_"+str(i+1)) for i in range(4)])

# Create a reverse id column
df = df.withColumn("id", row_number().over(w) - 1)
df = df.withColumn("id_2", df.count() - 1 - df["id"])

df_with_diag_zero = df.select([when(col("id_2") == i, 0).otherwise(col("col_"+str(i+1))).alias("col_"+str(i+1)) for i in range(4)])

df_with_diag_zero.show()

Py4JJavaError: An error occurred while calling o1304.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 196.0 failed 1 times, most recent failure: Lost task 0.0 in stage 196.0 (TID 459) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 47. How to reverse the rows of a dataframe?

In [44]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id

# Create a numeric DataFrame
data = [(1, 2, 3, 4),
(2, 3, 4, 5),
(3, 4, 5, 6),
(4, 5, 6, 7)]

df = spark.createDataFrame(data, ["col_1", "col_2", "col_3", "col_4"])

# Print DataFrame
df.show()

# Define window specification
w = Window.orderBy(monotonically_increasing_id())

# Add index
df = df.withColumn("id", row_number().over(w) - 1)

df_2 = df.orderBy("id", ascending=False).drop("id")

df_2.show()

Py4JJavaError: An error occurred while calling o1328.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 197.0 failed 1 times, most recent failure: Lost task 0.0 in stage 197.0 (TID 460) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 48. How to create one-hot encodings of a categorical variable (dummy variables)?

In [45]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
data = [("A", 10),("A", 20),("B", 30),("B", 20),("B", 30),("C", 40),("C", 10),("D", 10)]
columns = ["Categories", "Value"]

df = spark.createDataFrame(data, columns)
df.show()

indexer = StringIndexer(inputCol="Categories", outputCol="Categories_Indexed")
indexerModel = indexer.fit(df)
indexed_df = indexerModel.transform(df)

encoder = OneHotEncoder(inputCol="Categories_Indexed", outputCol="Categories_onehot")
encoded_df = encoder.fit(indexed_df).transform(indexed_df)
encoded_df = encoded_df.drop("Categories_Indexed")
encoded_df.show(truncate=False)

Py4JJavaError: An error occurred while calling o1352.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 198.0 failed 1 times, most recent failure: Lost task 0.0 in stage 198.0 (TID 461) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 49. How to Pivot the dataframe (converting rows into columns) ?

In [46]:
# Sample data
data = [
(2021, 1, "US", 5000),
(2021, 1, "EU", 4000),
(2021, 2, "US", 5500),
(2021, 2, "EU", 4500),
(2021, 3, "US", 6000),
(2021, 3, "EU", 5000),
(2021, 4, "US", 7000),
(2021, 4, "EU", 6000),
]

# Create DataFrame
columns = ["year", "quarter", "region", "revenue"]
df = spark.createDataFrame(data, columns)

# Execute the pivot operation
pivot_df = df.groupBy("year", "quarter").pivot("region").sum("revenue")

pivot_df.show()

Py4JJavaError: An error occurred while calling o1381.pivot.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 199.0 failed 1 times, most recent failure: Lost task 0.0 in stage 199.0 (TID 462) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.Iterab

## 50. How to UnPivot the dataframe (converting columns into rows) ?

In [47]:
from pyspark.sql.functions import expr
# Sample data
data = [(2021, 2, 4500, 5500),
(2021, 1, 4000, 5000),
(2021, 3, 5000, 6000),
(2021, 4, 6000, 7000)]

# Create DataFrame
columns = ["year", "quarter", "EU", "US"]
df = spark.createDataFrame(data, columns)

df.show()

unpivotExpr = "stack(2, 'EU',EU, 'US', US) as (region,revenue)"

unPivotDF = pivot_df.select("year","quarter", expr(unpivotExpr)).where("revenue is not null")

unPivotDF.show()

Py4JJavaError: An error occurred while calling o1405.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 200.0 failed 1 times, most recent failure: Lost task 0.0 in stage 200.0 (TID 468) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 51. How to impute missing values with Zero?

In [48]:
# Suppose df is your DataFrame
df = spark.createDataFrame([(1, None), (None, 2), (3, 4), (5, None)], ["a", "b"])

df.show()
df_imputed = df.fillna(0)

df_imputed.show()

Py4JJavaError: An error occurred while calling o1429.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 201.0 failed 1 times, most recent failure: Lost task 0.0 in stage 201.0 (TID 469) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 52. How to identify continuous variables in a dataframe and create a list of those column names?

In [49]:
url = "https://raw.githubusercontent.com/selva86/datasets/master/Churn_Modelling_m.csv"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("Churn_Modelling_m.csv"), header=True, inferSchema=True)


df.show(2, truncate=False)

from pyspark.sql.types import IntegerType, StringType, NumericType
from pyspark.sql.functions import approxCountDistinct

def detect_continuous_variables(df, distinct_threshold):
    continuous_columns = []
    for column in df.columns:
        dtype = df.schema[column].dataType
        if isinstance(dtype, (IntegerType, NumericType)):
            distinct_count = df.select(approxCountDistinct(column)).collect()[0][0]
        if distinct_count > distinct_threshold:
            continuous_columns.append(column)
    return continuous_columns

continuous_variables = detect_continuous_variables(df, 10)
print(continuous_variables)

Py4JJavaError: An error occurred while calling o1434.addFile.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1864)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1800)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:75)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMet

## 53. How to calculate Mode of a PySpark DataFrame column?

In [50]:
from pyspark.sql.functions import col
# Create a sample DataFrame
data = [(1, 2, 3), (2, 2, 3), (2, 2, 4), (1, 2, 3), (1, 1, 3)]
columns = ["col1", "col2", "col3"]

df = spark.createDataFrame(data, columns)

df.show()

df_grouped = df.groupBy('col2').count()
mode_df = df_grouped.orderBy(col('count').desc()).limit(1)

mode_df.show()

Py4JJavaError: An error occurred while calling o1457.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 202.0 failed 1 times, most recent failure: Lost task 0.0 in stage 202.0 (TID 470) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 54. How to find installed location of Apache Spark and PySpark?

In [51]:
import findspark
findspark.init()

print(findspark.find())

import os
import pyspark

print(os.path.dirname(pyspark.__file__))

c:\Users\jedre\anaconda3\envs\Python13\Lib\site-packages\pyspark
c:\Users\jedre\anaconda3\envs\Python13\Lib\site-packages\pyspark


## 55. How to convert a column to lower case using UDF?

In [52]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Create a DataFrame to test
data = [('John Doe', 'NEW YORK'),
('Jane Doe', 'LOS ANGELES'),
('Mike Johnson', 'CHICAGO'),
('Sara Smith', 'SAN FRANCISCO')]

df = spark.createDataFrame(data, ['Name', 'City'])

df.show()


# Define your UDF function
def to_lower(s):
    if s is not None:
        return s.lower()

# Convert your Python function to a Spark UDF
udf_to_lower = udf(to_lower, StringType())

# Apply your UDF to the DataFrame
df = df.withColumn('City_lower', udf_to_lower(df['City']))

# Show the DataFrame
df.show()

Py4JJavaError: An error occurred while calling o1481.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 203.0 failed 1 times, most recent failure: Lost task 0.0 in stage 203.0 (TID 471) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 56. How to convert PySpark data frame to pandas dataframe?

In [53]:
# Create a DataFrame to test
data = [('John Doe', 'NEW YORK'),
('Jane Doe', 'LOS ANGELES'),
('Mike Johnson', 'CHICAGO'),
('Sara Smith', 'SAN FRANCISCO')]

pysparkDF = spark.createDataFrame(data, ['Name', 'City'])

pysparkDF.show()

pandasDF = pysparkDF.toPandas()

print(pandasDF)

Py4JJavaError: An error occurred while calling o1505.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 204.0 failed 1 times, most recent failure: Lost task 0.0 in stage 204.0 (TID 472) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 57. How to View PySpark Cluster Details?

In [54]:
print(spark.sparkContext.uiWebUrl)

http://Jedrek:4040


## 58. How to View PySpark Cluster Configuration Details?

In [55]:
# Print all configurations
for k,v in spark.sparkContext.getConf().getAll():
    print(f"{k} : {v}")

spark.driver.port : 39838
spark.rdd.compress : True
spark.hadoop.fs.s3a.vectored.read.min.seek.size : 128K
spark.executor.extraJavaOptions : -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-modules=jdk.incubator.vector --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.refl

## 59. How to restrict the PySpark to use the number of cores in the system?

In [56]:
from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.set("spark.executor.cores", "2") # set the number of cores you want here

# Reuse existing SparkContext if present, otherwise create one with the given configuration
sc = SparkContext.getOrCreate(conf=conf)

## 60. How to cache PySpark DataFrame or objects and delete cache?

In [57]:
# Caching the DataFrame
df.cache()

# un-cache or unpersist data using the unpersist() method.
df.unpersist()

DataFrame[Name: string, City: string]

## 61. How to Divide a PySpark DataFrame randomly in a given ratio (0.8, 0.2)?

In [58]:
# Randomly split data (0.8, 0.2)
train_data, test_data = pysparkDF.randomSplit([0.8, 0.2], seed=42)


## 62. How to build logistic regression in PySpark?

In [59]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
# Create a sample dataframe
data = spark.createDataFrame([
(0, 1.0, -1.0),
(1, 2.0, 1.0),
(1, 3.0, -2.0),
(0, 4.0, 1.0),
(1, 5.0, -3.0),
(0, 6.0, 2.0),
(1, 7.0, -1.0),
(0, 8.0, 3.0),
(1, 9.0, -2.0),
(0, 10.0, 2.0),
(1, 11.0, -3.0),
(0, 12.0, 1.0),
(1, 13.0, -1.0),
(0, 14.0, 2.0),
(1, 15.0, -2.0),
(0, 16.0, 3.0),
(1, 17.0, -3.0),
(0, 18.0, 1.0),
(1, 19.0, -1.0),
(0, 20.0, 2.0)
], ["label", "feat1", "feat2"])



# convert the feature columns into a single vector column using VectorAssembler
vecAssembler = VectorAssembler(inputCols=['feat1', 'feat2'], outputCol="features")
data = vecAssembler.transform(data)

# fit the logistic regression model
lr = LogisticRegression(featuresCol='features', labelCol='label')
lr_model = lr.fit(data)

# look at the coefficients and intercept of the logistic regression model
print(f"Coefficients: {str(lr_model.coefficients)}")
print(f"Intercept: {str(lr_model.intercept)}")

Py4JJavaError: An error occurred while calling o1622.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 205.0 failed 1 times, most recent failure: Lost task 3.0 in stage 205.0 (TID 476) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.Iterable

## 63. How to convert the categorical string data into numerical data or index?


In [60]:
from pyspark.ml.feature import StringIndexer
# Create a sample DataFrame
data = [('cat',), ('dog',), ('mouse',), ('fish',), ('dog',), ('cat',), ('mouse',)]
df = spark.createDataFrame(data, ["animal"])

df.show()

# Initialize a StringIndexer
indexer = StringIndexer(inputCol='animal', outputCol='animalIndex')

# Fit the indexer to the DataFrame and transform the data
indexed = indexer.fit(df).transform(df)
indexed.show()

Py4JJavaError: An error occurred while calling o1685.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 207.0 failed 1 times, most recent failure: Lost task 0.0 in stage 207.0 (TID 479) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 64. How to calculate Correlation of two variables in a DataFrame?

In [61]:
# Create a sample dataframe
data = [Row(feature1=5, feature2=10, feature3=25),
Row(feature1=6, feature2=15, feature3=35),
Row(feature1=7, feature2=25, feature3=30),
Row(feature1=8, feature2=20, feature3=60),
Row(feature1=9, feature2=30, feature3=70)]
df = spark.createDataFrame(data)

df.show()
# Calculate correlation
correlation = df.corr("feature1", "feature2")

print("Correlation between feature1 and feature2 :", correlation)

Py4JJavaError: An error occurred while calling o1709.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 208.0 failed 1 times, most recent failure: Lost task 0.0 in stage 208.0 (TID 480) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 65. How to calculate Correlation Matrix?

In [62]:
from pyspark.ml.stat import Correlation
# Create a sample dataframe
data = [Row(feature1=5, feature2=10, feature3=25),
Row(feature1=6, feature2=15, feature3=35),
Row(feature1=7, feature2=25, feature3=30),
Row(feature1=8, feature2=20, feature3=60),
Row(feature1=9, feature2=30, feature3=70)]
df = spark.createDataFrame(data)

df.show()

# Calculate Correlation Using Using MLlib

# Assemble feature vector
# Define the feature and label columns & Assemble the feature vector
vector_assembler = VectorAssembler(inputCols=["feature1", "feature2", "feature3"], outputCol="features")
data_vector = vector_assembler.transform(df).select("features")

# Calculate correlation
correlation_matrix = Correlation.corr(data_vector, "features").head()[0]

print(correlation_matrix)

Py4JJavaError: An error occurred while calling o1733.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 209.0 failed 1 times, most recent failure: Lost task 0.0 in stage 209.0 (TID 481) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 66. How to calculate VIF (Variance Inflation Factor ) for set of variables in a DataFrame?

In [63]:
from pyspark.sql import SparkSession, Row
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# Create a sample dataframe
data = [Row(feature1=5, feature2=10, feature3=25),
Row(feature1=6, feature2=15, feature3=35),
Row(feature1=7, feature2=25, feature3=30),
Row(feature1=8, feature2=20, feature3=60),
Row(feature1=9, feature2=30, feature3=70)]
df = spark.createDataFrame(data)

df.show()


def calculate_vif(data, features):
    vif_dict = {}

    for feature in features:
        non_feature_cols = [col for col in features if col != feature]
        assembler = VectorAssembler(inputCols=non_feature_cols, outputCol="features")
        lr = LinearRegression(featuresCol='features', labelCol=feature)

        model = lr.fit(assembler.transform(data))
        vif = 1 / (1 - model.summary.r2)

        vif_dict[feature] = vif

        return vif_dict

features = ['feature1', 'feature2', 'feature3']
vif_values = calculate_vif(df, features)

for feature, vif in vif_values.items():
    print(f'VIF for {feature}: {vif}')

Py4JJavaError: An error occurred while calling o1757.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 210.0 failed 1 times, most recent failure: Lost task 0.0 in stage 210.0 (TID 482) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 67. How to perform Chi-Square test?


In [64]:
from pyspark.ml.feature import VectorAssembler
# Create a sample dataframe
data = [(1, 0, 0, 1, 1),
(2, 0, 1, 0, 0),
(3, 1, 0, 0, 0),
(4, 0, 0, 1, 1),
(5, 0, 1, 1, 0)]

df = spark.createDataFrame(data, ["id", "feature1", "feature2", "feature3", "label"])

df.show()

assembler = VectorAssembler(inputCols=["feature1", "feature2", "feature3"], outputCol="features")
df = assembler.transform(df)

from pyspark.ml.stat import ChiSquareTest

r = ChiSquareTest.test(df, "features", "label").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

Py4JJavaError: An error occurred while calling o1781.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 211.0 failed 1 times, most recent failure: Lost task 0.0 in stage 211.0 (TID 483) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 68. How to calculate the Standard Deviation?

In [65]:
from pyspark.sql.functions import stddev

# Sample data
data = [("James", "Sales", 3000),
("Michael", "Sales", 4600),
("Robert", "Sales", 4100),
("Maria", "Finance", 3000),
("James", "Sales", 3000),
("Scott", "Finance", 3300),
("Jen", "Finance", 3900),
("Jeff", "Marketing", 3000),
("Kumar", "Marketing", 2000),
("Saif", "Sales", 4100)]

# Create DataFrame
df = spark.createDataFrame(data, ["Employee", "Department", "Salary"])

df.show()

salary_stddev = df.select(stddev("Salary").alias("stddev"))

salary_stddev.show()

Py4JJavaError: An error occurred while calling o1805.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 212.0 failed 1 times, most recent failure: Lost task 0.0 in stage 212.0 (TID 484) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 69. How to calculate missing value percentage in each column?

In [66]:
# Create a sample dataframe
data = [("John", "Doe", None),
(None, "Smith", "New York"),
("Mike", "Smith", None),
("Anna", "Smith", "Boston"),
(None, None, None)]

df = spark.createDataFrame(data, ["FirstName", "LastName", "City"])

df.show()
# Calculate the total number of rows in the dataframe
total_rows = df.count()

# For each column calculate the number of null values and then calculate the percentage
for column in df.columns:
    null_values = df.filter(df[column].isNull()).count()
    missing_percentage = (null_values / total_rows) * 100
    print(f"Missing values in {column}: {missing_percentage}%")

Py4JJavaError: An error occurred while calling o1829.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 213.0 failed 1 times, most recent failure: Lost task 0.0 in stage 213.0 (TID 485) (Jedrek executor driver): java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1305)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1291)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:417)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1201)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1197)
	at scala.collection.I

## 70. How to get the names of DataFrame objects that have been created in an environment?

In [67]:
dataframe_names = [name for name, obj in globals().items() if isinstance(obj, pyspark.sql.DataFrame)]

for name in dataframe_names:
    print(name)

_
df
df_2
df_buck
df_filtered
df_A
df_B
summary
pivot_df
result
pysparkDF
_57
train_data
test_data
